In [ ]:
!pip install rouge-score bert-score
!pip install hf_xet

In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Importing Libraries - Downloading NLTK Resources and Loading Models

This code starts by importing several Python libraries that are essential for the task of text summarization and evaluation:

In [5]:
import json
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from transformers import pipeline
import spacy
import pandas as pd

nltk.download('wordnet')
# Load spaCy model for entity detection
nlp = spacy.load("en_core_web_sm")

# Load factuality QA model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Load JSON
with open('summarized_articles_pegasus-xsum.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

[nltk_data] Downloading package wordnet to /root/nltk_data...


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


## Evaluate Summaries

This function is designed to evaluate the quality of automatically generated text summaries by comparing them to reference summaries. It does this using a variety of metrics to assess different aspects of summary quality.

In [6]:
def evaluate_summaries(json_data, reference_keys, predicted_key,
    content_key="content", clean_content_key="clean_content"):
    """
    The function `evaluate_summaries` evaluates the quality of predicted summaries by comparing them
    with reference summaries using various metrics like BLEU, METEOR, ROUGE, BERTScore, and factual
    consistency.

    :param json_data: `json_data` is a list of dictionaries where each dictionary represents an article
    with keys like "title", "content", "clean_content", "predicted_summary", and reference summaries
    specified by `reference_keys`. The function `evaluate_summaries` evaluates the quality of predicted
    summaries against reference summaries for each article
    :param reference_keys: The `reference_keys` parameter in the `evaluate_summaries` function is a list
    of keys that represent the reference summaries in the JSON data. These keys are used to retrieve the
    reference summaries for comparison with the predicted summaries
    :param predicted_key: The `predicted_key` parameter in the `evaluate_summaries` function refers to
    the key in the JSON data that contains the predicted summary for each article. This key is used to
    extract the predicted summary for evaluation against the reference summaries
    :param content_key: The `content_key` parameter in the `evaluate_summaries` function refers to the
    key in the JSON data that contains the main content or text of an article. This content is used as
    the base for evaluating the predicted summaries against the reference summaries. If this key is not
    provided, an empty string, defaults to content (optional)
    :param clean_content_key: The `clean_content_key` parameter in the `evaluate_summaries` function is
    used to specify the key in the JSON data that contains the clean content of the article. This clean
    content is typically a version of the article content that has been processed or pre-processed to
    remove noise, special characters,, defaults to clean_content (optional)
    :return: The function `evaluate_summaries` returns a list of dictionaries containing evaluation
    metrics for each article-reference pair in the provided JSON data. The metrics include BLEU score,
    METEOR score, ROUGE scores (ROUGE-1, ROUGE-2, ROUGE-L), BERTScore F1 score, factual consistency
    score, number of hallucinated entities, and the list of hallucinated entities
    """

    all_metrics = []
    smoothie = SmoothingFunction().method4
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    for article_idx, article in enumerate(json_data):
        predicted = article.get(predicted_key)
        if not predicted:
            print(f"Skipping article {article_idx} (no predicted summary in '{predicted_key}')")
            continue

        content_text = article.get(content_key, "")
        clean_content = article.get(clean_content_key, "")

        # Hallucination detection
        doc = nlp(predicted)
        hallucinated_entities = [
            ent.text for ent in doc.ents if ent.text.lower() not in clean_content.lower()
        ]

        # QA-based factual consistency
        qa_results = []
        for chunk in doc.noun_chunks:
            question = f"What about {chunk.text}?"
            result = qa_pipeline(question=question, context=clean_content)
            qa_results.append({
                "question": question,
                "answer": result.get("answer", None),
                "score": result.get("score", 0),
                "is_answerable": result.get("answer", "") != ""
            })

        factual_score = (
            sum(1 for r in qa_results if r["is_answerable"]) / len(qa_results)
            if qa_results else None
        )

        for ref_name in reference_keys:
            ref = article.get(ref_name)
            if not ref:
                print(f"Article {article_idx}: missing reference '{ref_name}'")
                continue

            bleu = sentence_bleu([ref.split()], predicted.split(), smoothing_function=smoothie)
            meteor = meteor_score([ref.split()], predicted.split())
            rouge = scorer.score(ref, predicted)
            P, R, F1 = bert_score([predicted], [ref], lang="en", rescale_with_baseline=True)

            metrics = {
                "article_idx": article_idx,
                "reference_name": ref_name,
                "title": article.get("title"),
                "content": content_text,
                "clean_content": clean_content,
                "predicted_summary": predicted,
                "reference_summary": ref,
                "bleu": bleu,
                "meteor": meteor,
                "rouge1": rouge['rouge1'].fmeasure,
                "rouge2": rouge['rouge2'].fmeasure,
                "rougeL": rouge['rougeL'].fmeasure,
                "bertscore_f1": F1.item(),
                "factual_score": factual_score,
                "num_hallucinated_entities": len(hallucinated_entities),
                "hallucinated_entities": hallucinated_entities
            }

            all_metrics.append(metrics)

            print(f"\n--- Article {article_idx}, Reference {ref_name} ---")
            print(json.dumps(metrics, indent=2))

    # Optional aggregation
    if all_metrics:
        num_pairs = len(all_metrics)
        aggregate = {
            "bleu": sum(m["bleu"] for m in all_metrics) / num_pairs,
            "meteor": sum(m["meteor"] for m in all_metrics) / num_pairs,
            "rouge1": sum(m["rouge1"] for m in all_metrics) / num_pairs,
            "rouge2": sum(m["rouge2"] for m in all_metrics) / num_pairs,
            "rougeL": sum(m["rougeL"] for m in all_metrics) / num_pairs,
            "bertscore_f1": sum(m["bertscore_f1"] for m in all_metrics) / num_pairs,
            "factual_score": sum(m["factual_score"] for m in all_metrics if m["factual_score"] is not None) / num_pairs,
            "avg_hallucinated_entities": sum(m["num_hallucinated_entities"] for m in all_metrics) / num_pairs
        }

        print("\n=== Aggregated Metrics Across All Article-Reference Pairs ===")
        print(json.dumps(aggregate, indent=2))

    return all_metrics

In [11]:
reference_keys = ["summary_facebook/bart-large-cnn"]
predicted_key = "summary_finetuned_bart-large-cnn"

with open('summarized_articles_bart-large-cnn.json') as f:
    data = json.load(f)

metrics = evaluate_summaries(data, reference_keys, predicted_key)

# Optionally save to CSV
pd.DataFrame(metrics).to_csv("summary_metrics_bart-large-cnn.csv", index=False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- Article 0, Reference summary_facebook/bart-large-cnn ---
{
  "article_idx": 0,
  "reference_name": "summary_facebook/bart-large-cnn",
  "title": "US court eviscerates Apple\u2019s malicious compliance, claims company lied under oath several times",
  "content": "Way back in 2021, in the Epic v. Apple court case, judge US District Judge Yvonne Gonzalez Rogers ordered Apple to allow third-party developers to tell users how to make payments inside iOS applications without going through Apple\u2019s App Store. As we have come to expect from Apple, the company maliciously complied, lowering the commission on purchases outside of its ecosystem from 30% to 27%, while also adding a whole bunch of hoops and hurdles, like scare screens with doom-and-gloom language to, well, scare consumers into staying within Apple\u2019s ecosystem for in-app payments. Well, it turns out Judge Yvonne Gonzalez Rogers is furious, giving Apple, Tim Cook, and its other executives what can only be described as a

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- Article 3, Reference summary_facebook/bart-large-cnn ---
{
  "article_idx": 3,
  "reference_name": "summary_facebook/bart-large-cnn",
  "title": "Transcript: Lisa Shalett, CIO Morgan Stanley",
  "content": "",
  "clean_content": "The transcript from this week\u2019s, MiB: Lisa Shalett, CIO Morgan Stanley, is below. You can stream and download our full conversation, including any podcast extras, on Apple Podcasts, Spotify, YouTube, and Bloomberg. All of our earlier podcasts on your favorite\u2026",
  "predicted_summary": " The transcript from this week's, MiB: Lisa Shalett, CIO Morgan Stanley, is below . You can stream and download our full conversation, including any podcast extras, on Apple Podcasts, Spotify, YouTube, and Bloomberg.",
  "reference_summary": "The full transcript from this week's MiB: Lisa Shalett, CIO Morgan Stanley, is below. You can stream and download",
  "bleu": 0.32543875149111146,
  "meteor": 0.8042438428658661,
  "rouge1": 0.7547169811320755,
  "rouge2": 0.

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...


--- Article 30, Reference summary_facebook/bart-large-cnn ---
{
  "article_idx": 30,
  "reference_name": "summary_facebook/bart-large-cnn",
  "title": "Is the US pushing countries towards China?",
  "content": "By\u00a0 \n\n      Darian Woods\n    \n ,\u00a0 \n\n      Adrian Ma\n    \n ,\u00a0 \n\n      Lilly Quiroz\n    \n ,\u00a0 \n\n      Kate Concannon\n    \n As the U.S. goes head-to-head with the rest of the world on tariffs, those countries are trying to figure out their best diplomatic strategy. One dilemma countries have is how close they get with another global superpower: China. On today's show, we hear from Pakistan's Finance Minister Muhammad Aurangzeb about how the country is balancing trade relations with both countries. Related episodes: China's trade war perspective (Apple / Spotify) Dealmaker Don v. Tariff Man Trump (Apple / Spotify) Who's advising Trump on trade? (Apple / Spotify)  For sponsor-free episodes of The Indicator from Planet Money, subscribe to Planet Mon

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- Article 31, Reference summary_facebook/bart-large-cnn ---
{
  "article_idx": 31,
  "reference_name": "summary_facebook/bart-large-cnn",
  "title": "Here is how much US tech firms spent on lobbying the US government in 2024",
  "content": "30 minutes ago\n \n2 hours ago\n \n3 hours ago\n \n5 hours ago\n \n30 minutes ago\n \n2 hours ago\n \n3 hours ago\n \n14 hours ago\n \n5 hours ago\n \n7 hours ago\n \n19 hours ago\n \nMay 1, 2025\n \n3 hours ago\n \n19 hours ago\n \nMay 1, 2025\n \nMay 1, 2025\n \nMay 1, 2025\n \nMay 1, 2025\n \nApr 30, 2025\n \nApr 30, 2025\n \nApr 26, 2025\n \nApr 20, 2025\n \nApr 15, 2025\n \nApr 12, 2025\n \nApr 7, 2025\n \nMar 17, 2025\n \nFeb 1, 2025\n \nJan 18, 2025\n \nApr 21, 2025\n \nApr 21, 2025\n \nApr 20, 2025\n \nApr 17, 2025\n \nMar 5, 2025\n \nFeb 20, 2025\n \nFeb 19, 2025\n \nJan 30, 2025\n \n14 hours ago\n \n23 hours ago\n \n23 hours ago\n \nMay 1, 2025\n \nApr 30, 2025\n \nApr 27, 2025\n \nFeb 26, 2025\n \nFeb 15, 2025\n \n15 hours ago\n \n20 h

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- Article 32, Reference summary_facebook/bart-large-cnn ---
{
  "article_idx": 32,
  "reference_name": "summary_facebook/bart-large-cnn",
  "title": "Episode 5: Mom\u2019s at Work: The Price of Parenthood \u2013 why raising a family is more expensive than ever",
  "content": "9 spots left to attend the Digiday Programmatic Marketing Summit Subscribe: Apple Podcasts \u2022 Spotify For working moms, the rising cost of raising a family isn\u2019t just a financial burden \u2014 it\u2019s a daily source of stress that impacts careers, well-being, and the ability to plan for the future. In this episode, we dive into the rising financial pressures facing working parents today. From the skyrocketing costs of childcare, school supplies, summer camps, and college tuition to the everyday expenses of raising a family \u2013 the financial burden has never been higher.\u00a0 We speak with Kendra Cole, a PR professional and mother of three, about the crippling costs of summer camps across the U.S.